# FEDAVG BENCHMARK

We try to reproduce the results of the paper [Communication-Efficient Learning of Deep Networks from Decentralized Data](http://arxiv.org/abs/1602.05629) on the MNIST dataset. The paper introduces Federatad Learning as a concept with it's first implementation Federated Averaging. The algorithm is tested on the MNIST dataset with [MNIST_2NN](../fluke/nets.py:287)(a simple 2-layer perceptron).

## Setup of the experiment

### Loading and splitting the dataset

In [ ]:
from fluke.data.datasets import Datasets
dataset = Datasets.get("mnist", path="../data")

In [ ]:
from fluke.data import DataSplitter
splitter = DataSplitter(dataset=dataset,
                        distribution="iid",
                        client_split=0.2,
                        sampling_perc=1)

### Setting up the evaluator

In [ ]:
from fluke.evaluation import ClassificationEval, Evaluator
from fluke import GlobalSettings

evaluator = ClassificationEval(eval_every=1, n_classes=dataset.num_classes)
GlobalSettings().set_evaluator(evaluator)

### Setting up the hyperparameters and the model

In [ ]:
from fluke import DDict

client_hp = DDict(
    batch_size=10,
    local_epochs=5,
    loss="CrossEntropyLoss",
    optimizer=DDict(
      lr=0.1),
    scheduler=DDict(
      gamma=1,
      step_size=1)
)

alg_hp = DDict(
    client = client_hp,
    server=DDict(weighted=True),
    model="MNIST_2NN")

In [ ]:
from fluke.algorithms.fedavg import FedAVG
algorithm = FedAVG(100, splitter, alg_hp)

### Setting up the logger

In [ ]:
from fluke.utils.log import Log
logger = Log()
algorithm.set_callbacks(logger)

## Running the experiment

In [ ]:
algorithm.run(100, 0.2)